**Case 2:**

Target:

 * Lighter model

 * Reduce overfitting

 * Increase Model efficiency with Batch Normalization

 * Use GAP


Results:

~4k Parameters

Best Train Accuracy: 98.98

Best Test Accuracy: 98.9

Observations:

1. Model's parameters are brought down
2. Overfitting has reduced though not completely
3. Accuracy is still around 98



**Case 1:**

Target:

 * Basic Network. 

Results:

~16k Parameters

Best Train Accuracy: 99.26

Best Test Accuracy: 98.92

Observations:

1. Model has decent parameters
2. Overfitting can be seen happening after epoch 8

In [ ]:

from __future__ import print_function
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import itertools
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class Block(nn.Module):
    def __init__(self, input_size, output_size, padding=1, usepool=True):
        super(Block, self).__init__()
        self.usepool = usepool
        self.conv1 = nn.Conv2d(input_size, output_size, 3, padding=padding)
        self.bn1 = nn.BatchNorm2d(output_size)
        self.conv2 = nn.Conv2d(output_size, output_size, 3, padding=padding)
        self.bn2 = nn.BatchNorm2d(output_size)
        if usepool:
            self.pool = nn.MaxPool2d(2, 2)
        
    def __call__(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.bn1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.bn2(x)
        if self.usepool:
            x = self.pool(x)
        return x

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.base_channels = 4
        self.drop = 0.01

        # Conv 
        self.block1 = Block(1, self.base_channels)
        self.dropout1 = nn.Dropout(self.drop)
        self.block2 = Block(self.base_channels, self.base_channels*2)
        self.dropout2 = nn.Dropout(self.drop)
        self.block3 = Block(self.base_channels*2, self.base_channels*4, usepool=False)
        self.dropout3 = nn.Dropout(self.drop)

        self.flat = nn.Conv2d(self.base_channels*4, 10, 1)
        self.gap = nn.AdaptiveAvgPool2d(1)

    def forward(self, x, dropout=True):
        # Conv Layer
        x = self.block1(x)
        if dropout:
            x = self.dropout1(x)
        x = self.block2(x)
        if dropout:
            x = self.dropout2(x)
        x = self.block3(x)
        if dropout:
            x = self.dropout3(x)

        # Output Layer
        x = self.flat(x)
        x = self.gap(x)
        x = x.view(-1, 10)

        # Output Layer
        return F.log_softmax(x, dim=1)

In [ ]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 28, 28]              40
       BatchNorm2d-2            [-1, 4, 28, 28]               8
            Conv2d-3            [-1, 4, 28, 28]             148
       BatchNorm2d-4            [-1, 4, 28, 28]               8
         MaxPool2d-5            [-1, 4, 14, 14]               0
           Dropout-6            [-1, 4, 14, 14]               0
            Conv2d-7            [-1, 8, 14, 14]             296
       BatchNorm2d-8            [-1, 8, 14, 14]              16
            Conv2d-9            [-1, 8, 14, 14]             584
      BatchNorm2d-10            [-1, 8, 14, 14]              16
        MaxPool2d-11              [-1, 8, 7, 7]               0
          Dropout-12              [-1, 8, 7, 7]               0
           Conv2d-13             [-1, 16, 7, 7]           1,168
      BatchNorm2d-14             [-1, 1

In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation(5),      
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    train_correct = 0
    loss = 0
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True) 
        train_correct += pred.eq(target.view_as(pred)).sum().item()

    loss /= len(test_loader.dataset)

    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
    loss, train_correct, len(train_loader.dataset),
    100. * train_correct / len(train_loader.dataset)))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
no_of_epochs = 19
for epoch in range(no_of_epochs):
    print("Epoch Number :=> ",epoch+1)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch Number :=>  1

Train set: Average loss: 0.0000, Accuracy: 48670/60000 (81.12%)


Test set: Average loss: 0.1783, Accuracy: 9515/10000 (95.15%)

Epoch Number :=>  2

Train set: Average loss: 0.0000, Accuracy: 57807/60000 (96.34%)


Test set: Average loss: 0.0915, Accuracy: 9754/10000 (97.54%)

Epoch Number :=>  3

Train set: Average loss: 0.0000, Accuracy: 58428/60000 (97.38%)


Test set: Average loss: 0.0785, Accuracy: 9783/10000 (97.83%)

Epoch Number :=>  4

Train set: Average loss: 0.0000, Accuracy: 58697/60000 (97.83%)


Test set: Average loss: 0.0547, Accuracy: 9841/10000 (98.41%)

Epoch Number :=>  5

Train set: Average loss: 0.0000, Accuracy: 58841/60000 (98.07%)


Test set: Average loss: 0.0586, Accuracy: 9819/10000 (98.19%)

Epoch Number :=>  6

Train set: Average loss: 0.0000, Accuracy: 58957/60000 (98.26%)


Test set: Average loss: 0.0435, Accuracy: 9878/10000 (98.78%)

Epoch Number :=>  7

Train set: Average loss: 0.0000, Accuracy: 59001/60000 (98.33%)


Test set: Ave